### Upload data from Microsoft's Capstone Project DAT102x

Rate spread of mortgage loans in the US

Initialize Azure Machine Learning Workspace and authenticate

In [22]:
import logging
import os
import sys
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.pipeline.steps import AutoMLStep
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
# add common directory as module search path
common_path = os.getcwd()+"/../common"
if not common_path in sys.path:
    sys.path.append(common_path)
%load_ext autoreload
%autoreload 2
from ml_principal_authenticate import AzureMLAuthenticator

SDK version: 1.19.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
service_authenticator = AzureMLAuthenticator(config_path=os.path.normpath(f"{os.getcwd()}/../Config"))

ws = service_authenticator.get_workspace("aml_research")
if ws is not None:
    print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
else:
    print("Workspace not available")

aml_research
aml_research
westeurope
3cd9cbbe-bebe-4315-a11d-47eed87a8547


#### Upload the original dataset

This is the original, "dirty" dataset as it was provided by Microsoft for the Microsoft Professional Program. I want to compare the strength of Azure Machine Learning and how good it performs against the engineered dataset below I will upload later into which I invested several days to engineer 50 additional features, majorly statistics related to each data row.

In [21]:
import pandas as pd

# In the original dataset values and labels were split into separate files
# we combine them now again into a single Pandas dataframe
original_data = pd.read_csv('data/train_values.csv')
original_data_labels = pd.read_csv('data/train_labels.csv')
original_data['rate_spread'] = original_data_labels['rate_spread']
print(f"{len(original_data)} total rows")
original_data.head()

200000 total rows


,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,rate_spread
0,0,2,1,1,1,139.0,1,261,14,246,...,77.0,5949.0,11.642,63192.0,100.000,1556.0,1993.0,2094,True,1.0
1,1,2,2,1,1,160.0,3,349,48,311,...,42.0,2667.0,37.141,63693.0,75.729,875.0,1163.0,4194,False,2.0
2,2,1,2,1,1,168.0,3,385,30,256,...,34.0,6393.0,79.635,42883.0,76.948,636.0,1611.0,1119,False,3.0
3,3,1,2,1,1,57.0,3,117,30,46,...,37.0,7175.0,72.843,63806.0,78.836,1587.0,2172.0,1119,False,6.0
4,4,2,1,2,1,25.0,3,95,41,192,...,46.0,5041.0,23.188,70908.0,59.579,756.0,1992.0,1593,False,4.0


In [14]:
# Upload dataset to Azure
datastore = ws.get_default_datastore()
registered_set = TabularDatasetFactory.register_pandas_dataframe(original_data, datastore, "UncleanedMortgageSpread")

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/d576dc75-563d-4c32-bcf4-2a684c3fd0b2/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


#### Upload the engineered data set

This is the engineered dataset with cleaned rows, automatically filled missing values, attached statistics such as the average loan in the state, county or district etc.

In [18]:
import zipfile
import io

dataset_zip = zipfile.ZipFile("data/cleanedEngineeredData.zip", "r")
engineered_data = pd.read_csv(io.BytesIO(dataset_zip.read("train_cleaned.csv")))

In [25]:
print(f"{len(engineered_data)} total rows")
print(engineered_data.columns)
engineered_data.head()

200000 total rows
Index(['row_id', 'loan_type', 'property_type', 'loan_purpose', 'occupancy',
       'loan_amount', 'preapproval', 'msa_md', 'state_code', 'county_code',
       'applicant_ethnicity', 'applicant_race', 'applicant_sex',
       'applicant_income', 'population', 'minority_population_pct',
       'ffiecmedian_family_income', 'tract_to_msa_md_income_pct',
       'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
       'lender', 'co_applicant', 'rate_spread', 'sex_name', 'ethnicity_name',
       'race_name', 'loan_type_name', 'property_type_name', 'purpose_name',
       'tract_income', 'sex_male', 'sex_female', 'sex_other', 'purp_purchase',
       'purp_improvement', 'purp_refinancing', 'type_1_to_4', 'type_manuf',
       'type_multi_fam', 'oc_owner_occ', 'oc_not_owner_occ', 'oc_not_app',
       'preapp_owner_occ', 'preapp_not_oo', 'preapp_not_app', 'type_convl',
       'type_fha', 'type_va', 'type_fsarhs', 'race_white', 'race_black',
       'race_asian', 'ra

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,lender_spread_lt,lender_spread_lp,lender_spread_pt,county_spread,loantype_aspread,proptype_aspread,loanpurp_aspread,occupancy_aspread,state_spread,income_loan_rel
0,0,2,1,1,1,139.0,1,261,14,246,...,-0.742752,-0.708915,-0.689624,0.217119,-0.620380,-0.364393,-0.061103,-0.00892,0.250070,0.553957
1,1,2,2,1,1,160.0,3,349,48,311,...,-0.494021,-0.428029,-0.502672,0.564956,-0.620380,2.011958,-0.061103,-0.00892,0.133498,0.262500
2,2,1,2,1,1,168.0,3,385,30,256,...,3.721786,3.717423,3.721861,-0.534250,0.752205,2.011958,-0.061103,-0.00892,-0.127125,0.202381
3,3,1,2,1,1,57.0,3,117,30,46,...,3.721786,3.717423,3.721861,-0.040448,0.752205,2.011958,-0.061103,-0.00892,-0.127125,0.649123
4,4,2,1,2,1,25.0,3,95,41,192,...,1.275630,1.641015,1.425630,-0.406509,-0.620380,-0.364393,1.280664,-0.00892,-0.260579,1.840000


In [28]:
# Upload dataset to Azure
datastore = ws.get_default_datastore()
registered_set = TabularDatasetFactory.register_pandas_dataframe(engineered_data, datastore, "EngineeredMortgageSpread")

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/66430330-917a-4043-a0de-cbfa677c3af7/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [31]:
non_lender_spread = engineered_data.copy()
non_lender_spread = non_lender_spread.drop(columns=['lender_spread', 'lender_spread_lt','lender_spread_lp', 'lender_spread_pt'])
print(f"{len(non_lender_spread)} total rows")
print(non_lender_spread.columns)
non_lender_spread.head()

200000 total rows
Index(['row_id', 'loan_type', 'property_type', 'loan_purpose', 'occupancy',
       'loan_amount', 'preapproval', 'msa_md', 'state_code', 'county_code',
       'applicant_ethnicity', 'applicant_race', 'applicant_sex',
       'applicant_income', 'population', 'minority_population_pct',
       'ffiecmedian_family_income', 'tract_to_msa_md_income_pct',
       'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
       'lender', 'co_applicant', 'rate_spread', 'sex_name', 'ethnicity_name',
       'race_name', 'loan_type_name', 'property_type_name', 'purpose_name',
       'tract_income', 'sex_male', 'sex_female', 'sex_other', 'purp_purchase',
       'purp_improvement', 'purp_refinancing', 'type_1_to_4', 'type_manuf',
       'type_multi_fam', 'oc_owner_occ', 'oc_not_owner_occ', 'oc_not_app',
       'preapp_owner_occ', 'preapp_not_oo', 'preapp_not_app', 'type_convl',
       'type_fha', 'type_va', 'type_fsarhs', 'race_white', 'race_black',
       'race_asian', 'ra

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,sex_not_prov,income_not_prov,msa_spread,county_spread,loantype_aspread,proptype_aspread,loanpurp_aspread,occupancy_aspread,state_spread,income_loan_rel
0,0,2,1,1,1,139.0,1,261,14,246,...,False,0,0.502335,0.217119,-0.620380,-0.364393,-0.061103,-0.00892,0.250070,0.553957
1,1,2,2,1,1,160.0,3,349,48,311,...,False,0,0.144051,0.564956,-0.620380,2.011958,-0.061103,-0.00892,0.133498,0.262500
2,2,1,2,1,1,168.0,3,385,30,256,...,False,0,-0.534250,-0.534250,0.752205,2.011958,-0.061103,-0.00892,-0.127125,0.202381
3,3,1,2,1,1,57.0,3,117,30,46,...,False,0,-0.040448,-0.040448,0.752205,2.011958,-0.061103,-0.00892,-0.127125,0.649123
4,4,2,1,2,1,25.0,3,95,41,192,...,False,0,-0.270596,-0.406509,-0.620380,-0.364393,1.280664,-0.00892,-0.260579,1.840000


In [33]:
# Upload dataset to Azure
datastore = ws.get_default_datastore()
registered_set = TabularDatasetFactory.register_pandas_dataframe(non_lender_spread, datastore, "EngineeredMortgageSpreadNoLenderStats")

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/3e93b416-ba10-45c6-8da7-2f6e84bfdd53/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
